In [ ]:
movieimport numpy as np
import pandas as pd
import json
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from termcolor import colored
import ast

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading some amount of data 
lst = []
c=0
for row in open("/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json",'r'):
  c+=1
  data = json.loads(row)
  lst.append(data)
  if c==5000:
    break

In [ ]:
# Converting the input data to a Dataframe
df = pd.DataFrame(lst)
paper=df
paper.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [ ]:
# Taking only the useful columns
paper=paper[['id','authors','title','abstract']]
# for index,row in paper.iterrows():
#     print (index)

In [ ]:
# Converting all to lower case
paper.duplicated().sum()
paper['title'] = paper['title'].str.lower()
paper['authors'] = paper['authors'].str.lower()
 
paper['abstract'] = paper['abstract'].str.lower()
 
paper['tags'] =paper['title']+paper['abstract']
paper.head(5)

<ipython-input-7-d22d64444669>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paper['title'] = paper['title'].str.lower()
<ipython-input-7-d22d64444669>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paper['authors'] = paper['authors'].str.lower()
<ipython-input-7-d22d64444669>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,id,authors,title,abstract,tags
0,0704.0001,"c. bal\'azs, e. l. berger, p. m. nadolsky, c.-...",calculation of prompt diphoton production cros...,a fully differential calculation in perturba...,calculation of prompt diphoton production cros...
1,0704.0002,ileana streinu and louis theran,sparsity-certifying graph decompositions,"we describe a new algorithm, the $(k,\ell)$-...",sparsity-certifying graph decompositions we d...
2,0704.0003,hongjun pan,the evolution of the earth-moon system based o...,the evolution of earth-moon system is descri...,the evolution of the earth-moon system based o...
3,0704.0004,david callan,a determinant of stirling cycle numbers counts...,we show that a determinant of stirling cycle...,a determinant of stirling cycle numbers counts...
4,0704.0005,wael abu-shammala and alberto torchinsky,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,in this paper we show how to compute the $\l...,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...


In [ ]:
# paper=paper[['id','authors','title','tags']]
# paper.head(5)

In [ ]:
paper= paper[paper['tags'].notnull()]

# Splitting the tags
def convert(text):
    L = []
    counter = 0
    s=text.split(sep=None, maxsplit=-1)
    print(s)
    for i in s:
         if i[-1]==',' or i[-1]==';' or i[-1]=='.':
            i=i[:-1]
         
         L.append(s)
    return L 
paper['tags'] = paper['tags'].apply(convert) 
paper.head(5)

In [ ]:
# Further pre-processing
for index,row in paper.iterrows():
    L1=[]
    for i in paper.loc[index]['tags']:
        for j in i:
           L1.append(j)
    paper.loc[index]['tags']=L1
paper.head()  

,id,authors,title,abstract,tags
0,0704.0001,"c. bal\'azs, e. l. berger, p. m. nadolsky, c.-...",calculation of prompt diphoton production cros...,a fully differential calculation in perturba...,"[calculation, of, prompt, diphoton, production..."
1,0704.0002,ileana streinu and louis theran,sparsity-certifying graph decompositions,"we describe a new algorithm, the $(k,\ell)$-...","[sparsity-certifying, graph, decompositions, w..."
2,0704.0003,hongjun pan,the evolution of the earth-moon system based o...,the evolution of earth-moon system is descri...,"[the, evolution, of, the, earth-moon, system, ..."
3,0704.0004,david callan,a determinant of stirling cycle numbers counts...,we show that a determinant of stirling cycle...,"[a, determinant, of, stirling, cycle, numbers,..."
4,0704.0005,wael abu-shammala and alberto torchinsky,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,in this paper we show how to compute the $\l...,"[from, dyadic, $\lambda_{\alpha}$, to, $\lambd..."


In [ ]:
# Converting the list to a string in Tags column
s=''
for index,rows in paper.iterrows():
  paper.loc[index]['tags']=' '.join(paper.loc[index,'tags'])
paper.head()

,id,authors,title,abstract,tags
0,0704.0001,"c. bal\'azs, e. l. berger, p. m. nadolsky, c.-...",calculation of prompt diphoton production cros...,a fully differential calculation in perturba...,calculation of prompt diphoton production cros...
1,0704.0002,ileana streinu and louis theran,sparsity-certifying graph decompositions,"we describe a new algorithm, the $(k,\ell)$-...",sparsity-certifying graph decompositions we de...
2,0704.0003,hongjun pan,the evolution of the earth-moon system based o...,the evolution of earth-moon system is descri...,the evolution of the earth-moon system based o...
3,0704.0004,david callan,a determinant of stirling cycle numbers counts...,we show that a determinant of stirling cycle...,a determinant of stirling cycle numbers counts...
4,0704.0005,wael abu-shammala and alberto torchinsky,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,in this paper we show how to compute the $\l...,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...


In [ ]:
# ps=PorterStemmer()

In [ ]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    y = []
    for i in text.split():
        y.append(lemmatizer.lemmatize(i))

    return " ".join(y)

paper['tags']=paper['tags'].apply(lemmatize)
paper.head()

,id,authors,title,abstract,tags
0,0704.0001,"c. bal\'azs, e. l. berger, p. m. nadolsky, c.-...",calculation of prompt diphoton production cros...,a fully differential calculation in perturba...,calculation of prompt diphoton production cros...
1,0704.0002,ileana streinu and louis theran,sparsity-certifying graph decompositions,"we describe a new algorithm, the $(k,\ell)$-...",sparsity-certifying graph decomposition we des...
2,0704.0003,hongjun pan,the evolution of the earth-moon system based o...,the evolution of earth-moon system is descri...,the evolution of the earth-moon system based o...
3,0704.0004,david callan,a determinant of stirling cycle numbers counts...,we show that a determinant of stirling cycle...,a determinant of stirling cycle number count u...
4,0704.0005,wael abu-shammala and alberto torchinsky,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...,in this paper we show how to compute the $\l...,from dyadic $\lambda_{\alpha}$ to $\lambda_{\a...


In [ ]:
# Vectorizing using Count Vectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors_tag=cv.fit_transform(paper['tags']).toarray()
vectors_tag

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# Applying cosine similarity
similarity=cosine_similarity(vectors_tag)

In [ ]:
# Checking the cosine similraity for first index and taking the top 5 results
k=sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]
print(k)

[(4037, 0.3897164176493666), (1343, 0.33180130000452873), (263, 0.3138514904811289), (4038, 0.288377377801192), (4955, 0.27051435586009814)]


In [ ]:
print("Enter the query")
query=input()

Enter the query
differential calculus


In [ ]:
# Taking the query and preprocessing it

lemmatizer = WordNetLemmatizer()
tokens = nltk.word_tokenize(query)
lemmatized_query = [lemmatizer.lemmatize(token) for token in tokens]
lemmatized_sentence = " ".join(lemmatized_query)
print(lemmatized_sentence)


differential calculus


In [ ]:
# Vectroizing the query
query_vector = cv.transform([lemmatized_sentence]).toarray()
print(query_vector)

[[0 0 0 ... 0 0 0]]


In [ ]:
# Computing the cosine similarities between the query vector and the title vectors and showing the top 5 results
cos_similarities = cosine_similarity(query_vector, vectors_tag)

no_of_results = 5
similar_indices = cos_similarities.argsort()[0][-no_of_results:]

# Print the most similar titles and their cosine similarity scores

a=0
l_id=[]
l_title=[]
l_url=[]
for i in similar_indices:
    id=paper['id'][i]
    l_id.append(paper['id'][i])
    l_title.append(paper['title'][i])
    s="https://arxiv.org/pdf/"+paper['id'][i]
    l_url.append(s)

In [ ]:
print("Query is : "+query)
print()
print("The recommended papers are: ")
print()
from tabulate import tabulate
x=[]
for i in range(len(l_title)):
  l=[]
  l.append(l_title[i])
  l.append(l_url[i])
  x.append(l)


head=["Title","URL"]
print(tabulate(x, headers=head, tablefmt="grid"))


Query is : differential calculus

The recommended papers are: 

+-------------------------------------------------------------------+---------------------------------+
| Title                                                             | URL                             |
+===================================================================+=================================+
| a cut-free sequent calculus for bi-intuitionistic logic: extended | https://arxiv.org/pdf/0704.1707 |
|   version                                                         |                                 |
+-------------------------------------------------------------------+---------------------------------+
| bringing errors into focus                                        | https://arxiv.org/pdf/0705.0519 |
+-------------------------------------------------------------------+---------------------------------+
| on differential sequences                                         | https://arxiv.org/pdf/0704.3243 |


# Text Summarization

In [ ]:
pip install spellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for spellchecker: filename=spellchecker-0.4-py3-none-any.whl size=3966516 sha256=34ae43b9bf55deaf66e272fac2cb4002f812ef44373e869708d82c4a3cda30fd
  Stored in directory: /root/.cache/pip/wheels/8c/52/b3/8795c86fe999a3b1a7d0f1b75a197fd984e37057cb537e2977
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7141 sha256=8a4ab8ad62c6e1244168923e2a8fe5b8f9fc7d191c107abe66514c382b03b9f7
  Stored in directory: /root/.cache/pip/wheels/da/15/83/b2b0adf6ebb17b0c7b8de4a901f0585ceb7627788a80071de0
  Created wheel for silpa_common: filename=silpa_common-0.3-py3-none-any.whl size=8480 sha256=62d4a60bcdf91ce2970654a50

In [ ]:
pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26528019 sha256=49f78c5cc994bfd8dc1bd9659958d1cb3470b0af3c5e67ffbec985e7adb1069f
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


In [1]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [2]:
import re
import os
from PyPDF2 import PdfReader
# from transformers import pipeline 
import pandas as pd

import requests
#from gensim.summarization.summarizer import summarize
#from gensim.summarization import keywords
import en_core_web_sm
import warnings
warnings.filterwarnings('ignore')

In [3]:
import shutil
shutil.rmtree('/content/download_data/', ignore_errors=True)
os.mkdir("/content/download_data/")

In [ ]:
# # Downloading the file chosen by the user
# def download_pdf_file(url: str) -> bool:
#   response = requests.get(url, stream=True)
#   pdf_file_name = os.path.basename(url)
#   if response.status_code == 200:
#     filepath = os.path.join("/content/download_data", pdf_file_name)
#     with open(filepath, 'wb') as pdf_object:
#       pdf_object.write(response.content)
#       print(f'{pdf_file_name} was successfully saved!')
#       return True
#   else:
#     print(f'Uh oh! Could not download {pdf_file_name},')
#     print(f'HTTP response status code: {response.status_code}')
#     return False

# if __name__ == '__main__':
#   URL = 'https://arxiv.org/pdf/0704.0047.pdf'
#   download_pdf_file(URL)

In [ ]:
# Reading the contents of the paper's pdf
final = []
files = os.listdir('/content/download_data/')
c=0
for file in files:
  lst = []
  print(file)
  if c<2:
    pdf_reader = PdfReader('/content/download_data/'+file)
    num_pages = len(pdf_reader.pages)
    content = ""
    for page in range(num_pages):
        pdf_page = pdf_reader.pages[page]
        text = pdf_page.extract_text()
        content += text
    lst.append(content)
    final.append(lst)
content

0705.1309.pdf


'arXiv:0705.1309v1  [cs.AI]  9 May 2007Robust Multi-CellularDevelopmentalDesign\nto appear in D. Thierens et al., Eds., Proceedings of GECCO’0 7, ACM Press, July 2007\nAlexandre Devert\nTAO/LRI ;INRIAFuturs,\nCNRS, Univ Paris-Sud\nF-91405,France\ndevert@lri.frNicolas Bredeche\nTAO /LRI;Univ Paris-Sud,\nCNRS,INRIA Futurs\nF-91405,France\nbredeche@lri.frMarc Schoenauer\nTAO /LRI;INRIA Futurs,\nCNRS,Univ Paris-Sud\nF-91405,France\nmarc@lri.fr\nABSTRACT\nThis paper introduces a continuous model for Multi-cellula r\nDevelopmental Design. The cells are ﬁxed on a 2D grid\nand exchange ”chemicals” with their neighbors during the\ngrowth process. The quantity of chemicals that a cell pro-\nduces, as well as the diﬀerentiation value of the cell in the\nphenotype, are controlled by a Neural Network (the geno-\ntype) that takes as inputs the chemicals produced by the\nneighboring cells at the previous time step. In the proposed\nmodel, the number of iterations of the growth process is not\npre-det

In [ ]:
import spacy
import re

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Open the research paper
# with open("research_paper.txt") as f:
#     text = f.read()

# Parse the text using Spacy
doc = nlp(content)
print(doc)
# Extract the section names
sections = []
for sent in doc.sents:
    # Check if the sentence is a section heading
    if re.match(r"^[A-Z][A-Za-z\s]+[A-Z]$", sent.text):
        sections.append(sent.text.strip())

# Print the section names
print(sections)

In [ ]:
pip install textract

In [ ]:
import spacy
import re
import os
import textract
import requests
from bs4 import BeautifulSoup

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Function to extract text from a file
def extract_text(filepath):
    # Extract text from PDF or Word document
    if filepath.endswith(".pdf") or filepath.endswith(".doc") or filepath.endswith(".docx"):
        text = textract.process(filepath).decode("utf-8")
    # Extract text from HTML document
    elif filepath.endswith(".html") or filepath.endswith(".htm"):
        html = requests.get(filepath).content
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text()
    # Extract text from plain text file
    else:
        with open(filepath) as f:
            text = f.read()
    print(text)
    return text

# Function to extract section headings from text
# def extract_section_headings(text):
#     # Parse the text using Spacy
#     doc = nlp(text)
#     # Extract the section headings
#     section_headings = []
#     for token in doc:
#         if token.is_title:
#             section_headings.append(token.text)
#     # Remove duplicates and return the section headings
#     return list(set(section_headings))

# Example usage
filepath = "/content/download_data/0705.1309.pdf"
text = extract_text(filepath)
# section_headings = extract_section_headings(text)
# print(section_headings)


In [ ]:
pattern = r"\n(.+)\n"
matches = re.findall(pattern, text)
f=[]
print(len(matches))
for j in range(len(matches)):
  if "Abstract" in matches[j] or "ABSTRACT" in matches[j]:
    break
for i in range(j,len(matches)):
  
  s=matches[i].split()
  #print(s)
  if len(s)<4:
    # print("length",end="  ")
    # print(len(s),matches[i])
    f.append(matches[i])
  
print(len(f))
print(f)

863
353
['Abstract', '1', 'Introduction', 'nlpyang/PreSumm.', 'paragraph-level natural', '2 Background', '(1)', 'contextual information.', 'speciﬁc parameters.', '2.2 Extractive Summarization', '2.3 Abstractive Summarization', 'distributions (TCONVS2S).', 'Original BERT', 'BERT for Summarization', 'Input ', 'Token ', 'Segment ', 'Position ', 'Contextual ', '[CLS]', 'sent', 'one', '[SEP]', '2nd', 'sent', '[SEP]', 'sent', 'again [SEP]', '[CLS]', 'sent', 'one', '[SEP]', '[CLS]', '2nd', 'sent', '[SEP]', '[CLS]', 'sent', 'again', '[SEP]', 'E[CLS] Esent', 'Eone E[SEP] E2nd', 'EA', 'E1', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'E6', 'E8', 'E9', 'E2', 'E3', 'E4', 'E5', 'E7', 'E2nd', 'E[CLS] Esent', 'Eone E[SEP] E[CLS]', 'EA', 'E1', 'E11', 'E10', 'EA', 'EA', 'EB', 'EB', 'EA', 'EA', 'EB', 'EB', 'EA', 'EA', 'E7', 'E9', 'E2', 'E4', 'E6', 'E8', 'E3', 'E5', 'Transformer Layers', 'Transformer Layers', 'T[CLS]', 'Tsent', 'Tone', 'T[SEP]', 'T2nd', 'Tsent', 'T[SEP]', 'Tsent Tagain T[SEP]', 'T[CL

In [ ]:
f2=[]
for i in range(len(f)):
 

  if f[i][0].islower()==False:
      # print("lower",end="  ")
      # print(len(s),i)
      #print(f[i])
      f2.append(f[i])

print(len(f),len(f2))
print(f2)

353 266
['Abstract', '1', 'Introduction', '2 Background', '(1)', '2.2 Extractive Summarization', '2.3 Abstractive Summarization', 'Original BERT', 'BERT for Summarization', 'Input ', 'Token ', 'Segment ', 'Position ', 'Contextual ', '[CLS]', '[SEP]', '2nd', '[SEP]', '[CLS]', '[SEP]', '[CLS]', '2nd', '[SEP]', '[CLS]', '[SEP]', 'E[CLS] Esent', 'Eone E[SEP] E2nd', 'EA', 'E1', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'EA', 'E6', 'E8', 'E9', 'E2', 'E3', 'E4', 'E5', 'E7', 'E2nd', 'E[CLS] Esent', 'Eone E[SEP] E[CLS]', 'EA', 'E1', 'E11', 'E10', 'EA', 'EA', 'EB', 'EB', 'EA', 'EA', 'EB', 'EB', 'EA', 'EA', 'E7', 'E9', 'E2', 'E4', 'E6', 'E8', 'E3', 'E5', 'Transformer Layers', 'Transformer Layers', 'T[CLS]', 'Tsent', 'Tone', 'T[SEP]', 'T2nd', 'Tsent', 'T[SEP]', 'Tsent Tagain T[SEP]', 'T[CLS]', 'Tsent', 'Tone', 'T[SEP] T[CLS]', 'T2nd', 'Tsent', 'T[SEP] T[CLS]', 'Tsent', 'Tagain T[SEP]', '3.2 Extractive Summarization', 'It', '(3)', 'ˆyi = σ(WohL', '(5)', 'In', 'BERTSUMEXT.', '3.3 Abstractive Summari

In [ ]:
f3=[]
for i in range(len(f2)):
  if f2[i].isdigit()==False:
    #print(i)
    f3.append(f2[i])
print(len(f2),len(f3))


266 264


In [ ]:
f4=[]
for i in range(len(f3)):
  if len(f3[i])>3:
    #print(i)
    f4.append(f3[i])
print(len(f3),len(f4))

264 193


In [ ]:
def isfloat(num):
  try:
    float(num)
    return True
  except ValueError:
    return False

f5=[]
for i in range(len(f4)):
  if isfloat(f4[i])==False:
    #print(i)
    f5.append(f4[i])
print(len(f4),len(f5))

193 147


In [ ]:
print(f5)

['Abstract', 'Introduction', '2 Background', '2.2 Extractive Summarization', '2.3 Abstractive Summarization', 'Original BERT', 'BERT for Summarization', 'Input ', 'Token ', 'Segment ', 'Position ', 'Contextual ', '[CLS]', '[SEP]', '[SEP]', '[CLS]', '[SEP]', '[CLS]', '[SEP]', '[CLS]', '[SEP]', 'E[CLS] Esent', 'Eone E[SEP] E2nd', 'E2nd', 'E[CLS] Esent', 'Eone E[SEP] E[CLS]', 'Transformer Layers', 'Transformer Layers', 'T[CLS]', 'Tsent', 'Tone', 'T[SEP]', 'T2nd', 'Tsent', 'T[SEP]', 'Tsent Tagain T[SEP]', 'T[CLS]', 'Tsent', 'Tone', 'T[SEP] T[CLS]', 'T2nd', 'Tsent', 'T[SEP] T[CLS]', 'Tsent', 'Tagain T[SEP]', '3.2 Extractive Summarization', 'ˆyi = σ(WohL', 'BERTSUMEXT.', '3.3 Abstractive Summarization', '\x0cDatasets', 'DailyMail', 'XSum', '# docs (train/val/test)', '196,961/12,148/10,397', '204,045/11,332/11,334', '4 Experimental Setup', '4.1 Summarization Datasets', 'Implementation Details', '2https://git.io/fhbJQ', 'Model', 'LEAD-3', '52.59 31.24 48.87', 'Extractive', 'TransformerEXT', 'A

In [ ]:
f6=[]
for i in range(len(f5)):
  my_new_string = re.sub('[^.a-zA-Z0-9 \n\.]', '', f5[i])
  if len(f5[i])==len(my_new_string):
    f6.append(f5[i])
print(len(f5),len(f6))

147 76


In [ ]:
print(f6)

['Abstract', 'Introduction', '2 Background', '2.2 Extractive Summarization', '2.3 Abstractive Summarization', 'Original BERT', 'BERT for Summarization', 'Input ', 'Token ', 'Segment ', 'Position ', 'Contextual ', 'E2nd', 'Transformer Layers', 'Transformer Layers', 'Tsent', 'Tone', 'T2nd', 'Tsent', 'Tsent', 'Tone', 'T2nd', 'Tsent', 'Tsent', '3.2 Extractive Summarization', 'BERTSUMEXT.', '3.3 Abstractive Summarization', 'DailyMail', 'XSum', '4 Experimental Setup', '4.1 Summarization Datasets', 'Implementation Details', 'Model', '52.59 31.24 48.87', 'Extractive', 'TransformerEXT', 'Abstractive', '36.44 15.66 33.42', '39.87 15.82 36.90', '41.69 19.47 37.92', 'BERTSUMEXT', 'BERTSUMABS', '42.13 19.60 39.18', '5 Results', 'Extractive', 'Abstractive', 'BERTSUMEXT', 'BERTSUMEXTABS', 'Model', 'LEAD', '8.81 22.66', 'Abstractive', '9.21 23.24', '31.89 11.54 25.75', 'BERTSUMABS', '38.76 16.33 31.15', '5.2 Model Analysis', '5.3 Human Evaluation', 'TRANSFORMEREXT', 'ORACLE', 'LEAD', 'BOTTOMUP', 'GOLD

In [ ]:
# import re

# my_str = "u' %$HI# Jhon, $how$ are *&$%you.%$"
# my_new_string = re.sub('[^.a-zA-Z0-9 \n\.]', '', my_str)
# print (my_new_string)

u HI Jhon how are you.


In [ ]:
import spacy
import re
import os
import PyPDF2

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract section headings
def extract_section_headings(filepath):
    # Open the PDF file in binary mode
    with open(filepath, "rb") as f:
        # Create a PyPDF2 reader object
        pdf_reader = PyPDF2.PdfReader(f)

        # Read in the contents of the PDF file
        contents = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            contents += page.extract_text()

    # Tokenize the file contents using Spacy
    doc = nlp(contents)

    # Extract the section headings
    section_headings = []
    for i, token in enumerate(doc):
        # Check if the token is a section heading
        if token.is_title and (i == 0 or doc[i-1].is_punct or doc[i-1].is_space):
            # Extract the section heading text
            heading_text = token.text.strip()
            # Remove any trailing punctuation
            heading_text = re.sub(r"\p{P}+$", "", heading_text)
            # Append the section heading to the list
            section_headings.append(heading_text)

    # Return the list of section headings
    return section_headings

# Example usage
filepath = "/content/download_data/0705.1309.pdf"
section_headings = extract_section_headings(filepath)
print(section_headings)


error: ignored

In [ ]:
# Converting to dataframe
final_df = pd.DataFrame(final,columns=["text"])
final_df

,text
0,arXiv:0705.1309v1 [cs.AI] 9 May 2007Robust M...


In [ ]:
content = final_df["text"]

summ_per = content.apply(summarize,ratio = 0.8)
summ_words = content.apply(summarize, word_count = 1000)

final_df['summ_per'] = summ_per
final_df['summ_words'] = summ_words
final_df.head()

In [ ]:
s=str(summ_per)
print(len(s))

NameError: ignored

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
ptorch1 = summarizer(s, min_length=20,max_length=100)

print(ptorch1)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Your max_length is set to 100, but you input_length is only 21. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


[{'summary_text': '0 Intelligent location of simultaneously active ... Name: text, dtype: object .'}]


In [ ]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import xml.etree.ElementTree as ET
# Open the PDF file
pdf_file = open('0705.1309.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
text = ""
num_pages = len(pdf_reader.pages)
print(num_pages)
for i in range(num_pages):
  page = pdf_reader.pages[i]
  text += page.extract_text()

# Create an XML document
root = ET.Element('document')
for line in text.split('\n'):
  if line.strip() != '':
    ET.SubElement(root, 'line').text = line

xml_file = open('example.xml', 'wb')
s=ET.tostring(root)
xml_file.write(ET.tostring(root))
xml_file.close()

8


In [ ]:
l

In [ ]:
# summarizer = pipeline("summarization", model="longformer-base-4096")
# print(summarizer(intro, max_length=200, min_length=100, do_sample=False))